In [1]:
from os import listdir, path
from numpy import array
from keras import initializers
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical, Sequence
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adamax
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np

from keras import backend as K
from nltk.translate.bleu_score import corpus_bleu

Using TensorFlow backend.


In [ ]:
K.tensorflow_backend._get_available_gpus()

In [2]:
dir_name = 'resources/eval/'

# Read a file and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []
    # Load all the files and order them
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "npz":
            # Load the images already prepared in arrays
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

train_features, texts = load_data(dir_name)

In [3]:
# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
tokenizer.fit_on_texts([load_doc('resources/bootstrap.vocab')])

# Add one spot for the empty word in the vocabulary 
vocab_size = len(tokenizer.word_index) + 1
# Map the input sentences into the vocabulary indexes
train_sequences = tokenizer.texts_to_sequences(texts)
# The longest set of boostrap tokens
max_sequence = max(len(s) for s in train_sequences)
# Specify how many tokens to have in each input sentence
max_length = 48

def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(X), np.array(y), np.array(image_data)
    """
    base_dir = dir_name.split('/')[0]
    data_dir = dir_name.split('/')[1]
    subpath_name = base_dir + '/' + data_dir
    if not path.exists(subpath_name + 'X.npy'): 
        np.save(subpath_name  + 'X.npy', np.array(X))
    if not path.exists(subpath_name + 'y.npy'):
        np.save(subpath_name + 'y.npy', np.array(y))
    if not path.exists(subpath_name + 'image_data.npy'):
        np.save(subpath_name + 'image_data.npy', np.array(image_data))
    """

X, y, image_data = preprocess_data(train_sequences, train_features)
#preprocess_data(train_sequences, train_features)

In [ ]:
# X, y, image_data
# X.shape = (*,48)
# y.shape = (*,18)
# image_data.shape = (*,256,256,3)

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, 
                 n_seqlen=48, n_onehot=18, img_dim=(256,256,3), 
                 shuffle=True):
        'Initialization'
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.n_seqlen = n_seqlen
        self.n_onehot = n_onehot
        self.dim = img_dim
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y, image_data = self.__data_generation(list_IDs_temp)

        return X, y, image_data

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, self.n_seqlen))
        y = np.empty((self.batch_size, self.n_onehot), dtype=int)
        image_data = np.empty((self.batch_size, *self.dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sequence
            X[i,] = np.load('dir_name' + ID + 'X.npy')

            # Store predicted one-hot encoding
            y[i] = np.load('dir_name' + ID + 'y.npy')
            
            # Store image
            image_data[i] = np.load('dir_name' + ID + 'image_data.npy')

        return X, to_categorical(y, num_classes=self.n_onehot), image_data

In [ ]:
# Parameters
params = {'batch_size': 10,
          'n_seqlen': max_length,
          'n_onehot': vocab_size,
          'img_dim': (256,256,3),
          'shuffle': True}

# Datasets
#partition = [image_data, X]
#labels = y

# Generators
#training_generator = DataGenerator(partition['train'], labels, **params)
#validation_generator = DataGenerator(partition['validation'], labels, **params)

In [ ]:
#Create the encoder
image_model = Sequential()
image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(256, 256, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)

#Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate a description for an image
def generate_desc(loaded_model, tokenizer, photo, max_length):
    photo = np.array([photo])
    # seed the generation process
    in_text = '<START> '
    # iterate over the whole length of the sequence
    #print('\nPrediction---->\n\n<START> ', end='')
    for i in range(150):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = loaded_model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += word + ' '
        # stop if we predict the end of the sequence
        #print(word + ' ', end='')
        if word == '<END>':
            break
    return in_text

class BleuCallback(Callback):
    max_length = 48
    
    def __init__(self, texts, train_features, tokenizer):
        self.texts = texts
        self.train_features = train_features
        self.tokenizer = tokenizer
        return
        
    # evaluate the skill of the model
    def evaluate_model(self, model, texts, photos, tokenizer, max_length):
    #def evaluate_model(model, texts, photos, tokenizer, max_length):
        actual, predicted = list(), list()
        # step over the whole set
        for i in range(len(texts)):
            yhat = generate_desc(model, tokenizer, photos[i], max_length)
            # store actual and predicted
            #print('\n\nReal---->\n\n' + texts[i])
            actual.append([texts[i].split()])
            predicted.append(yhat.split())
        # calculate BLEU score
        bleu = corpus_bleu(actual, predicted)
        print('\nBleu Score: {}\n'.format(bleu))
        
    def on_epoch_end(self, epoch, logs={}):
        self.evaluate_model(self.model, self.texts, self.train_features, self.tokenizer, max_length)
        return

In [13]:
#Save the model for every 2nd epoch
filepath="org-weights-epoch-{epoch:04d}--val_loss-{val_loss:.4f}--loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=1)
# Tensorboard
board = TensorBoard(log_dir='./logs', histogram_freq=10, batch_size=10, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# Bleu Score
bleu = BleuCallback(texts, train_features, tokenizer)
callbacks_list = [checkpoint, board, bleu]

In [ ]:
# Train the model
history = model.fit([image_data, X], y, batch_size=10, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=50)
"""
model.fit_generator(generator=training_generator
                    validation_data=validation_generator, 
                    callbacks=None,
                    use_multiprocessing=False,
                    workers=1,
                    shuffle=True,
                    verbose=1,
                    epochs=20)
"""

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['training loss', 'validation loss'], loc='upper left')
plt.show()